In [1]:
from pathlib import Path
import pickle
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D
import seaborn as sns
sns.set_theme()
import math
import pandas as pd
import random
import numpy as np
import sys
import shutil

import IMP
import IMP.atom

sys.path.append(str(Path(Path.home(), "Documents/xray/src")))

In [3]:
from iotbx import pdb
from iotbx.pdb.hierarchy import root

pdb_file_1 = Path(Path.home(), "Documents/xray/dev/45_synthetic_native_4/data/pdbs/native.pdb")
pdb_file_2 = Path(Path.home(), "Documents/xray/data/pdbs/3k0m/3k0m.pdb")

In [ ]:
multi_xray_h = root()

i = 0
for pdb_file in [pdb_file_1, pdb_file_2]:
    pdb_inp = pdb.input(file_name=str(pdb_file))
    hierarchy = pdb_inp.construct_hierarchy()

    asc = hierarchy.atom_selection_cache()
    sel = asc.selection("not altloc 'B'")

    hierarchy = hierarchy.select(sel)

    models = hierarchy.models()
    for model in models:
        model.id = str(i+1)
        multi_xray_h.append_model(model.detached_copy())

        i += 1


In [17]:
print(multi_xray_h.as_pdb_string())

MODEL        1
ATOM      1  N   VAL A   2     -19.567  -3.203 -17.428  1.00 15.00           N
ATOM      2  CA  VAL A   2     -19.242  -1.880 -17.955  1.00 15.00           C
ATOM      3  C   VAL A   2     -18.168  -1.238 -17.129  1.00 15.00           C
ATOM      4  O   VAL A   2     -17.279  -1.907 -16.654  1.00 15.00           O
ATOM      5  CB  VAL A   2     -20.491  -0.986 -18.010  1.00 15.00           C
ATOM      6  HA  VAL A   2     -18.896  -1.971 -18.854  1.00 15.00           H
ATOM      7  HB  VAL A   2     -21.205  -1.467 -18.461  1.00 15.00           H
ATOM      8  N   ASN A   3     -18.327   0.051 -16.888  1.00 15.00           N
ATOM      9  CA  ASN A   3     -17.173   0.867 -16.639  1.00 15.00           C
ATOM     10  C   ASN A   3     -16.545   1.240 -17.979  1.00 15.00           C
ATOM     11  O   ASN A   3     -16.209   0.355 -18.778  1.00 15.00           O
ATOM     12  CB  ASN A   3     -16.159   0.125 -15.764  1.00 15.00           C
ATOM     13  CG  ASN A   3     -16.72

In [42]:
from iotbx import pdb

# Assuming you have a hierarchy object
# If you have a PDB file, read it and construct the hierarchy
pdb_inp = pdb.input(file_name=str(Path(Path.home(), "Documents/xray/data/pdbs/3k0m/3k0m.pdb")))
hierarchy = pdb_inp.construct_hierarchy()

asc = hierarchy.atom_selection_cache()

# Create a selection for atoms where altloc is not 'B'
sel = asc.selection("not altloc 'B'")

print(hierarchy.atoms().size())

hierarchy = hierarchy.select(sel)

print(hierarchy.atoms().size())

# # Define your target atom name and residue name
# atom_name = 'HB2'  # Replace 'X' with the atom name you're interested in (e.g., 'CA')
# residue_name = 'Y'  # Replace 'Y' with the residue name (e.g., 'ALA')

# # Create an atom selection cache
# asc = hierarchy.atom_selection_cache()

# # Construct the selection string
# selection_string = f"resseq 16 and name HB2"

# # Create the selection
# sel = asc.selection(selection_string)

# # Apply the selection to get the selected atoms
# selected_atoms = hierarchy.select(sel).atoms()

# # Now you can work with the selected atoms
# for atom in selected_atoms:
#     atom_group = atom.parent()
#     residue_group = atom_group.parent()
#     chain = residue_group.parent()

#     print(f"""
#     Atom Name     : {atom.name.strip()}
#     Element       : {atom.element.strip()}
#     Residue Name  : {atom_group.resname.strip()}
#     Chain ID      : {chain.id.strip()}
#     Residue Number: {residue_group.resseq.strip()}
#     Coordinates   : {atom.xyz}
#     """)

3006
2733


In [43]:

crystal_symmetry = pdb_inp.crystal_symmetry()

xray_structure = hierarchy.extract_xray_structure(crystal_symmetry=crystal_symmetry)

print(xray_structure.scatterers()[0].site)
print(xray_structure.scatterers()[0].occupancy)

(-0.45093000777953274, -0.05912139309251853, -0.21016708220986252)
1.0


In [44]:
atom = hierarchy.atoms()[0]

print(atom.name)
print(atom.occ)
print(atom.xyz)

atom.occ = 0.5
atom.xyz = (0, 0, 0)

print(atom.occ)
print(atom.xyz)


 N  
1.0
(-19.128, -3.059, -18.629)
0.5
(0.0, 0.0, 0.0)


In [45]:
print(xray_structure.scatterers()[0].site)
print(xray_structure.scatterers()[0].occupancy)

# Update xray_structure
xray_structure.set_sites_cart(hierarchy.atoms().extract_xyz())
xray_structure.set_occupancies(hierarchy.atoms().extract_occ())

print(xray_structure.scatterers()[0].site)
print(xray_structure.scatterers()[0].occupancy)

(-0.45093000777953274, -0.05912139309251853, -0.21016708220986252)
1.0
(0.0, 0.0, 0.0)
0.5


In [60]:
occs = xray_structure.scatterers().extract_occupancies()
print(type(occs))
xray_structure.set_occupancies(occs)

<class 'scitbx_array_family_flex_ext.double'>


  xray.structure(
    crystal_symmetry=crystal.symmetry(
      unit_cell=(42.419, 51.741, 88.639, 90, 90, 90),
      space_group_symbol="P 21 21 21"
    ),
    scatterers=flex.xray_scatterer([
      xray.scatterer( #0
        label='pdb=" N   VAL A   2 "',
        scattering_type="N",
        site=(0.000000, 0.000000, 0.000000),
        u=(0.000280, 0.000203, 0.000082,
           -0.000081, -0.000021, 0.000000),
        occupancy=0.500000),
      xray.scatterer( #1
        label='pdb=" CA  VAL A   2 "',
        scattering_type="C",
        site=(-0.469059, -0.035214, -0.210302),
        u=(0.000243, 0.000193, 0.000075,
           -0.000063, -0.000012, -0.000005),
        occupancy=0.500000),
      xray.scatterer( #2
        label='pdb=" C   VAL A   2 "',
        scattering_type="C",
        site=(-0.449351, -0.012408, -0.204853),
        u=(0.000180, 0.000191, 0.000055,
           -0.000103, -0.000020, -0.000028),
        occupancy=0.500000),
      xray.scatterer( #3
        label='pdb

In [16]:
occupancies = hierarchy.atoms().extract_occ()
# Update xray_structure
xray_structure = xray_structure.set_occupancies(occupancies)

In [20]:
print(occupancies[0])
print(atom.occupancy)

1.0
0.5


In [17]:
print(xray_structure.scatterers()[0].site)
print(xray_structure.scatterers()[0].occupancy)

(-0.45093000777953274, -0.05912139309251853, -0.21016708220986252)
1.0


In [10]:
crystal_symmetry = pdb_inp.crystal_symmetry()

xray_structure = hierarchy.extract_xray_structure(crystal_symmetry=crystal_symmetry)

print(xray_structure.scatterers()[0].label)
print(xray_structure.scatterers()[0].site)
print(xray_structure.scatterers()[0].occupancy)

pdb=" N   VAL A   2 "
(0.0, 0.0, 0.0)
1.0


In [9]:
print(atom.occupancy)

0.5


In [60]:
# Create an atom selection cache
asc = hierarchy.atom_selection_cache()

# Construct the selection string
selection_string = f"resseq 2 and name N"

# Create the selection
sel = asc.selection(selection_string)

# Apply the selection to get the selected atoms
selected_atoms = hierarchy.select(sel).atoms()

atom = selected_atoms[0]

# # Now you can work with the selected atoms
# for atom in selected_atoms:
#     atom_group = atom.parent()
#     residue_group = atom_group.parent()
#     chain = residue_group.parent()

#     print(f"""
#     Atom Name     : {atom.name.strip()}
#     Element       : {atom.element.strip()}
#     Residue Name  : {atom_group.resname.strip()}
#     Chain ID      : {chain.id.strip()}
#     Residue Number: {residue_group.resseq.strip()}
#     Coordinates   : {atom.xyz}
#     """)

In [66]:
atom.xyz = (5,5,5)
print(atom.name, atom.xyz)

 N   (5.0, 5.0, 5.0)


In [65]:
test_dict = {}
test_dict[atom] = 1

In [67]:
test_dict[atom]

1

In [44]:

crystal_symmetry = pdb_inp.crystal_symmetry()

xray_structure = hierarchy.extract_xray_structure(crystal_symmetry=crystal_symmetry)

In [45]:
print(xray_structure.scatterers()[0].site)
print(xray_structure.scatterers()[0].label)

(-0.45093000777953274, -0.05912139309251853, -0.21016708220986252)
pdb=" N   VAL A   2 "


In [26]:
m = IMP.Model()
h = IMP.atom.read_pdb(str(Path(Path.home(), "Documents/xray/data/pdbs/3k0m/3k0m.pdb")), m, IMP.atom.AllPDBSelector())

begin read_pdb:
  WARNING  Could not determine CHARMM atom type for atom "HET: O  " in residue #312 "HOH"
end read_pdb


In [30]:
for pid in IMP.atom.Selection(h).get_selected_particles():
    at = IMP.atom.Atom(pid)

    res = IMP.atom.Residue(at.get_parent())

    print(at.get_name(), at.get_atom_type(), res.get_index())

Atom N of residue 2 "N" 2
Atom CA of residue 2 "CA" 2
Atom C of residue 2 "C" 2
Atom O of residue 2 "O" 2
Atom CB of residue 2 "CB" 2
Atom HA of residue 2 "HA" 2
Atom HB of residue 2 "HB" 2
Atom N of residue 3 "N" 3
Atom CA of residue 3 "CA" 3
Atom C of residue 3 "C" 3
Atom O of residue 3 "O" 3
Atom CB of residue 3 "CB" 3
Atom CG of residue 3 "CG" 3
Atom OD1 of residue 3 "OD1" 3
Atom ND2 of residue 3 "ND2" 3
Atom H of residue 3 "H" 3
Atom HA of residue 3 "HA" 3
Atom HB2 of residue 3 "HB2" 3
Atom HB3 of residue 3 "HB3" 3
Atom HD21 of residue 3 "HD21" 3
Atom HD22 of residue 3 "HD22" 3
Atom N of residue 4 "N" 4
Atom CA of residue 4 "CA" 4
Atom C of residue 4 "C" 4
Atom O of residue 4 "O" 4
Atom CB of residue 4 "CB" 4
Atom CG of residue 4 "CG" 4
Atom CD of residue 4 "CD" 4
Atom HA of residue 4 "HA" 4
Atom HB2 of residue 4 "HB2" 4
Atom HB3 of residue 4 "HB3" 4
Atom HG2 of residue 4 "HG2" 4
Atom HG3 of residue 4 "HG3" 4
Atom HD2 of residue 4 "HD2" 4
Atom HD3 of residue 4 "HD3" 4
Atom N of re